In [2]:
!pip install git+https://github.com/fkunstner/dataset-downloader.git

  Cloning https://github.com/fkunstner/dataset-downloader.git to /tmp/pip-req-build-jzzl_m1v
  Running command git clone --filter=blob:none --quiet https://github.com/fkunstner/dataset-downloader.git /tmp/pip-req-build-jzzl_m1v
  Resolved https://github.com/fkunstner/dataset-downloader.git to commit 44e1689ee298d6fa62b7cf6097b39656447e244a
  Preparing metadata (setup.py) ... done
  Created wheel for dsdl: filename=dsdl-0.1.2-py3-none-any.whl size=7783 sha256=e52e5ddc68934ae5b0090c653d8a5147ac8d9e6053348cdb9d69a435eb533ab7
  Stored in directory: /tmp/pip-ephem-wheel-cache-1qx2u24o/wheels/84/de/37/0632ccca850b40d000d4874635fd16e9eee004dd28fbb6d1d1
Successfully built dsdl


In [3]:
#Attribute Inference Attack
#Attacker assumptions: The synthetic data generated by the CTGAN is published, as it is anonymous and can be made publicly available, being GDPR compliant, and the attacker has access to a subset of original attributes

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import dsdl
import statistics
import sklearn
from sklearn.decomposition import PCA
print(dsdl.available_datasets())

['a1a', 'a2a', 'a3a', 'a4a', 'a5a', 'a6a', 'a7a', 'a8a', 'a9a', 'australian', 'australian_scale', 'breast-cancer', 'breast-cancer_scale', 'cod-rna', 'colon-cancer', 'covtype.binary', 'covtype.binary.scale', 'diabetes', 'diabetes_scale', 'duke-breast-cancer', 'fourclass', 'fourclass_scale', 'german.numer', 'german.numer_scale', 'gisette', 'heart', 'heart_scale', 'ijcnn1', 'ionosphere', 'leukemia', 'madelon', 'mushrooms', 'news20.binary', 'phishing', 'rcv1.binary', 'real-sim', 'skin_nonskin', 'splice', 'sonar', 'svmguide1', 'svmguide3', 'w1a', 'w2a', 'w3a', 'w4a', 'w5a', 'w6a', 'w7a', 'w8a', 'abalone', 'abalone_scale', 'bodyfat', 'bodyfat_scale', 'cadata', 'cpusmall', 'cpusmall_scale', 'SVHN', 'cifar10', 'criteo', 'webspam', 'kdd2010raw', 'E2006-log1p', 'E2006-E2006-tfidf', 'eunite2001', 'mg', 'mg_scale', 'mpg', 'mpg_scale', 'pyrim', 'pyrim_scale', 'space_ga', 'space_ga_scale', 'triazines', 'triazines_scale', 'YearPredictionMSD', 'yacht', 'boston-housing', 'california-housing', 'concrete

In [4]:
ds = dsdl.load("diabetes")

X, y = ds.get_train()

    Extracting /root/data/dsdl/diabetes/diabetes to /root/data/dsdl/diabetes
    - Nothing to extract in /root/data/dsdl/diabetes/diabetes


In [5]:
data = pd.DataFrame(X.todense())
data

,0,1,2,3,4,5,6,7
0,6.0,148.0,72.0,35.0,0.0,33.599998,0.627,50.0
1,1.0,85.0,66.0,29.0,0.0,26.600000,0.351,31.0
2,8.0,183.0,64.0,0.0,0.0,23.299999,0.672,32.0
3,1.0,89.0,66.0,23.0,94.0,28.100000,0.167,21.0
4,0.0,137.0,40.0,35.0,168.0,43.099998,2.288,33.0
...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.900002,0.171,63.0
764,2.0,122.0,70.0,27.0,0.0,36.799999,0.340,27.0
765,5.0,121.0,72.0,23.0,112.0,26.200001,0.245,30.0
766,1.0,126.0,60.0,0.0,0.0,30.100000,0.349,47.0


In [6]:
labels = pd.DataFrame(y)
data_with_labels = pd.concat([data,labels], axis=1)

In [7]:
data_with_labels.columns = [i for i in range(len(data_with_labels.columns))]

In [8]:
data_with_labels

,0,1,2,3,4,5,6,7,8
0,6.0,148.0,72.0,35.0,0.0,33.599998,0.627,50.0,-1.0
1,1.0,85.0,66.0,29.0,0.0,26.600000,0.351,31.0,1.0
2,8.0,183.0,64.0,0.0,0.0,23.299999,0.672,32.0,-1.0
3,1.0,89.0,66.0,23.0,94.0,28.100000,0.167,21.0,1.0
4,0.0,137.0,40.0,35.0,168.0,43.099998,2.288,33.0,-1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.900002,0.171,63.0,1.0
764,2.0,122.0,70.0,27.0,0.0,36.799999,0.340,27.0,1.0
765,5.0,121.0,72.0,23.0,112.0,26.200001,0.245,30.0,1.0
766,1.0,126.0,60.0,0.0,0.0,30.100000,0.349,47.0,-1.0


In [9]:
data_with_labels[8].value_counts()[-1]

268

In [10]:
data_with_labels[8].value_counts()[1]

500

In [11]:
!pip install CTGAN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.5 MB/s eta 0:00:00


In [13]:
from ctgan import CTGAN
from ctgan import load_demo

In [14]:
discrete_columns= [0,8]
ctgan = CTGAN(epochs=10)
ctgan.fit(data_with_labels.values, discrete_columns)
#generated 10 times the samples of the original data
synthetic_data = ctgan.sample(7680)

In [15]:
synthetic_data = pd.DataFrame(synthetic_data)

In [16]:
attacker_data1 = data.iloc[:,1:5]

In [17]:
synthetic_data1 = synthetic_data.iloc[:,1:5]

In [18]:
from scipy.spatial.distance import cdist
distance = cdist(synthetic_data1, attacker_data1, 'euclid' )
distance.shape

(7680, 768)

In [19]:
distance_df = pd.DataFrame(distance)

In [20]:
minvalueIndexLabel = distance_df.idxmin(axis = 1)

minvalueIndexLabel

0       584
1       346
2       254
3       729
4       724
       ... 
7675     13
7676    247
7677    584
7678     28
7679    681
Length: 7680, dtype: int64

In [21]:
synthetic_data['min_index'] = minvalueIndexLabel

In [22]:
count_number_of_times_pregnant = 0
for i in range(len(synthetic_data.index)):
    j=0
    row_value = synthetic_data['min_index'][i]
    if abs(data_with_labels[j][row_value]-synthetic_data[j][i])<=5:
        count_number_of_times_pregnant+=1

In [23]:
count_number_of_times_pregnant/len(synthetic_data.index)

0.5490885416666667

In [27]:
count_age = 0
for i in range(len(synthetic_data.index)):
    j=7
    row_value = synthetic_data['min_index'][i]
    if abs((data_with_labels[j][row_value]-synthetic_data[j][i])/data_with_labels[j][row_value])<=0.3:
        count_age+=1


In [28]:
count_age/len(synthetic_data.index)

0.49075520833333336

In [29]:
count_age = 0
for i in range(len(synthetic_data.index)):
    j=7
    row_value = synthetic_data['min_index'][i]
    if abs((data_with_labels[j][row_value]-synthetic_data[j][i])/data_with_labels[j][row_value])<=0.5:
        count_age+=1


In [30]:
count_age/len(synthetic_data.index)

0.7079427083333333

In [31]:
count_diabetes = 0
for i in range(len(synthetic_data.index)):
    j=8
    row_value = synthetic_data['min_index'][i]
    if data_with_labels[j][row_value]==synthetic_data[j][i]:
        count_diabetes+=1

In [32]:
count_diabetes/len(synthetic_data.index)

0.5055989583333333